In [1]:
import sys
import os

import torchaudio.transforms as T
import pandas                as pd
import numpy                 as np

from torchvision       import transforms

sys.path.append(os.path.abspath(os.path.join('..','..','iESPnet_SRC_main','utilities')))
from Generator         import SeizureDatasetLabelTime, smoothing_label
from Model             import iESPnet
from TrainEval         import test_model_iespnet, get_performance_indices

In [2]:
# direccion donde se encuentran los espectrogramas 
SPE_DIR        = '/media/martin/Disco2/Rns_Data/PITT_PI_SPEC/'
meta_data_file = '/media/martin/Disco2/Rns_Data/PITT_PI_SPEC/METADATA/allfiles_metadata.csv'

df_meta        = pd.read_csv(meta_data_file)

In [3]:
# Variables iESPnet
FREQ_MASK_PARAM       = 10
TIME_MASK_PARAN       = 20
N_CLASSES             = 1
learning_rate_iespnet = 1e-3
batch_size            = 64    #128
epochs                = 20
num_workers           = 4

In [5]:
patients = df_meta['rns_id'].unique().tolist()

In [6]:
# hiperparametros iESPnet y DSF
hparams = {
           "n_cnn_layers"          : 3,
           "n_rnn_layers"          : 3,
           "rnn_dim"               : [150, 100, 50],
           "n_class"               : N_CLASSES,
           "out_ch"                : [8,8,16],
           "dropout"               : 0.3,
           "learning_rate_iespnet" : learning_rate_iespnet,
           "batch_size"            : batch_size,
           "num_workers"           : num_workers,
           "epochs"                : epochs
          }

In [8]:
model = iESPnet(
                hparams['n_cnn_layers'],
                hparams['n_rnn_layers'],
                hparams['rnn_dim'],
                hparams['n_class'],
                hparams['out_ch'],
                hparams['dropout'],
               )

In [9]:
save_path        = '/media/martin/Disco2/Rns_Data/experimentos/iespnet_lopo/'
#save_models      = save_path + patients[s] + '/models/'
#save_predictions = save_path + patients[s] + '/results/'

best_thr         = 0.2

In [12]:
for s in range (len(patients)):

    # define test de df_meta
    test_df  = df_meta[df_meta['rns_id'] == patients[s]].copy()      
    test_df.reset_index(drop = True, inplace=True)

    test_epochs = test_df['data'].apply(lambda x: x.split("_")[2]).unique()
    
    save_runs        = save_path + patients[s] + '/runs/'
    save_models      = save_path + patients[s] + '/models/'
    save_predictions = save_path + patients[s] + '/results/' 
    save_figs        = save_path + patients[s] + '/figs/'

    if not os.path.exists(save_path):
        os.makedirs(save_path)

    if not os.path.exists(save_runs):
        os.makedirs(save_runs)

    if not os.path.exists(save_models):
        os.makedirs(save_models)

    if not os.path.exists(save_predictions):
        os.makedirs(save_predictions)

    if not os.path.exists(save_figs):
        os.makedirs(save_figs)
    
    best_path  = save_models + 'model.pth'

    for epoch_counter in test_epochs:
                          
        test_df_pe = test_df[test_df['data'].apply(lambda x: x.split("_")[2]) == epoch_counter]
                
        test_data_pe = SeizureDatasetLabelTime(
                                               file             = test_df_pe,
                                               root_dir         = SPE_DIR,
                                               transform        = None,
                                               target_transform =smoothing_label()  
                                              )
    
        print()
        print('in test: ',patients[s])
        # in test
        outputs_test  = test_model_iespnet(model, hparams, best_path, test_data_pe)
        prediction_te = get_performance_indices(outputs_test['y_true'], outputs_test['y_prob'], best_thr)

        predict_ = { 
                    "prediction_te"   : prediction_te,
                    }

        np.save(save_predictions + str(epoch_counter) + '_' + 'results.npy', predict_)

        del test_data_pe


in test:  PIT-RNS1603
Using cuda device
Testing process has finished.
Accuracy : 0.96 F1 : 0.49 Precission : 0.50 Recall : 0.48 

in test:  PIT-RNS1603
Using cuda device


/home/martin/Documentos/environments/env_thalamus/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2458: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


Testing process has finished.
Accuracy : 0.49 F1 : 0.49 Precission : 0.49 Recall : 0.49 

in test:  PIT-RNS1603
Using cuda device
Testing process has finished.
Accuracy : 0.81 F1 : 0.71 Precission : 0.63 Recall : 0.81 MAEtime : 5.67

in test:  PIT-RNS1603
Using cuda device
Testing process has finished.
Accuracy : 0.63 F1 : 0.58 Precission : 0.54 Recall : 0.63 MAEtime : 2.33

in test:  PIT-RNS1603
Using cuda device
Testing process has finished.
Accuracy : 0.80 F1 : 0.66 Precission : 0.56 Recall : 0.80 MAEtime : 1.80

in test:  PIT-RNS1603
Using cuda device
Testing process has finished.
Accuracy : 0.95 F1 : 0.85 Precission : 0.76 Recall : 0.95 MAEtime : 1.74

in test:  PIT-RNS1603
Using cuda device
Testing process has finished.
Accuracy : 0.92 F1 : 0.83 Precission : 0.75 Recall : 0.92 MAEtime : 1.64

in test:  PIT-RNS1603
Using cuda device
Testing process has finished.
Accuracy : 0.91 F1 : 0.89 Precission : 0.86 Recall : 0.91 MAEtime : 3.55

in test:  PIT-RNS1603
Using cuda device
Testin

/home/martin/Documentos/environments/env_thalamus/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2458: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


Testing process has finished.
Accuracy : 0.98 F1 : 0.95 Precission : 0.92 Recall : 0.98 MAEtime : 1.30

in test:  PIT-RNS6989
Using cuda device
Testing process has finished.
Accuracy : 0.75 F1 : 0.43 Precission : 0.50 Recall : 0.38 

in test:  PIT-RNS6989
Using cuda device


...

/home/martin/Documentos/environments/env_thalamus/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2458: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


Testing process has finished.
Accuracy : 0.93 F1 : 0.94 Precission : 0.95 Recall : 0.93 MAEtime : 4.90

in test:  PIT-RNS2368
Using cuda device
Testing process has finished.
Accuracy : 0.98 F1 : 0.89 Precission : 0.81 Recall : 0.98 MAEtime : 0.42

in test:  PIT-RNS2368
Using cuda device
Testing process has finished.
Accuracy : 0.95 F1 : 0.88 Precission : 0.82 Recall : 0.95 MAEtime : 3.61

in test:  PIT-RNS2368
Using cuda device
Testing process has finished.
Accuracy : 0.99 F1 : 0.97 Precission : 0.96 Recall : 0.99 MAEtime : 0.50

in test:  PIT-RNS2368
Using cuda device
Testing process has finished.
Accuracy : 0.97 F1 : 0.94 Precission : 0.91 Recall : 0.97 MAEtime : 0.82

in test:  PIT-RNS2368
Using cuda device
Testing process has finished.
Accuracy : 1.00 F1 : 1.00 Precission : 1.00 Recall : 1.00 MAEtime : 0.00

in test:  PIT-RNS6762
Using cuda device
Testing process has finished.
Accuracy : 0.85 F1 : 0.46 Precission : 0.50 Recall : 0.43 

in test:  PIT-RNS6762
Using cuda device


/home/martin/Documentos/environments/env_thalamus/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2458: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


Testing process has finished.
Accuracy : 0.79 F1 : 0.86 Precission : 0.93 Recall : 0.79 MAEtime : 5.18

in test:  PIT-RNS6762
Using cuda device
Testing process has finished.
Accuracy : 0.61 F1 : 0.61 Precission : 0.62 Recall : 0.61 MAEtime : 11.67

in test:  PIT-RNS2938
Using cuda device
Testing process has finished.
Accuracy : 0.90 F1 : 0.67 Precission : 0.53 Recall : 0.90 MAEtime : 5.50

in test:  PIT-RNS2938
Using cuda device
Testing process has finished.
Accuracy : 0.85 F1 : 0.82 Precission : 0.80 Recall : 0.85 MAEtime : 4.92

in test:  PIT-RNS2938
Using cuda device
Testing process has finished.
Accuracy : 0.90 F1 : 0.67 Precission : 0.54 Recall : 0.90 MAEtime : 12.25

in test:  PIT-RNS2938
Using cuda device
Testing process has finished.
Accuracy : 0.94 F1 : 0.78 Precission : 0.67 Recall : 0.94 MAEtime : 2.00

in test:  PIT-RNS2938
Using cuda device
Testing process has finished.
Accuracy : 0.89 F1 : 0.70 Precission : 0.58 Recall : 0.89 MAEtime : 5.36

in test:  PIT-RNS2938
Using cu

/home/martin/Documentos/environments/env_thalamus/lib/python3.10/site-packages/sklearn/metrics/_classification.py:2458: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


In [ ]:
for epoch_counter in test_epochs:

    # define a test_df_pe
    # the programmming epoch information is in test_df.data
    # "data" column is the file name for the spectrogram
    test_df_pe = test_df[test_df['epoch'] == epoch_counter]

    test_data_pe = SeizureDatasetLabelTime(file=test_df_pe,
                        root_dir=SPE_DIR,
                        transform=None,
                        target_transform=smoothing_label()  
                        )